In [2]:
import os
import sys
import argparse
import time
import numpy as np
import cv2
import torch
import torch.nn.functional as F
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime

# Add model path
sys.path.append('model')
sys.path.append('.')
from model.architecture import IMDN

In [3]:
SCALE = 2
MODEL_DIR = "/home/kronbii/repos/thermal-super-resolution/checkpoints/thermal_x2/thermal_best.pth"
TEST_DIR = f"./datasets/flir_thermal_x{SCALE}/val/LR_bicubic/X{SCALE}"
GT_DIR = f"./datasets/flir_thermal_x{SCALE}/val/HR"
OUTPUT_DIR = "./results/my_thermal_test_run" 
SAVE_RESULTS = True
DEVICE = "cuda"
NUM_SAMPLES = 1100

In [4]:
def load_thermal_image(image_path):
    """Load and preprocess thermal image"""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Cannot load image: {image_path}")
    
    # Convert to float32 and normalize to [0, 1]
    img = img.astype(np.float32) / 255.0
    
    # Add batch and channel dimensions
    img = torch.from_numpy(img).unsqueeze(0).unsqueeze(0)
    return img

In [5]:
def calculate_psnr(img1, img2, max_val=1.0):
    """Calculate PSNR between two images"""
    mse = torch.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * torch.log10(max_val / torch.sqrt(mse))

In [6]:
def calculate_ssim(img1, img2, max_val=1.0, window_size=11):
    """Calculate SSIM between two images"""
    def gaussian_window(size, sigma=1.5):
        coords = torch.arange(size, dtype=torch.float32)
        coords -= size // 2
        g = torch.exp(-(coords**2) / (2 * sigma**2))
        g /= g.sum()
        return g

    def create_window(window_size, channel=1):
        _1D_window = gaussian_window(window_size)
        _2D_window = _1D_window.unsqueeze(1) * _1D_window.unsqueeze(0)
        _2D_window = _2D_window.unsqueeze(0).unsqueeze(0)
        return _2D_window.expand(channel, 1, window_size, window_size).contiguous()

    if img1.shape != img2.shape:
        return 0.0

    window = create_window(window_size, img1.size(1)).to(img1.device)
    
    mu1 = F.conv2d(img1, window, padding=window_size//2, groups=img1.size(1))
    mu2 = F.conv2d(img2, window, padding=window_size//2, groups=img2.size(1))

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=window_size//2, groups=img1.size(1)) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=window_size//2, groups=img2.size(1)) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=window_size//2, groups=img2.size(1)) - mu1_mu2

    C1 = (0.01 * max_val)**2
    C2 = (0.03 * max_val)**2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

In [7]:
def save_comparison_image(lr_img, sr_img, gt_img, save_path, scale):
    """Save comparison of LR, SR, and GT images"""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Convert tensors to numpy for display
    lr_np = lr_img.squeeze().cpu().numpy()
    sr_np = sr_img.squeeze().cpu().numpy()
    
    # Display LR image
    axes[0].imshow(lr_np, cmap='hot', vmin=0, vmax=1)
    axes[0].set_title(f'Original Low-Res\n{lr_np.shape[0]}×{lr_np.shape[1]} pixels', fontsize=12, weight='bold')
    axes[0].axis('off')
    
    # Display SR image
    axes[1].imshow(sr_np, cmap='hot', vmin=0, vmax=1)
    axes[1].set_title(f'AI Enhanced ({scale}x)\n{sr_np.shape[0]}×{sr_np.shape[1]} pixels', fontsize=12, weight='bold', color='green')
    axes[1].axis('off')
    
    # Display GT image if available
    if gt_img is not None:
        gt_np = gt_img.squeeze().cpu().numpy()
        axes[2].imshow(gt_np, cmap='hot', vmin=0, vmax=1)
        axes[2].set_title(f'Ground Truth\n{gt_np.shape[0]}×{gt_np.shape[1]} pixels', fontsize=12, weight='bold')
    else:
        # Show bicubic upsampling as reference
        lr_bicubic = F.interpolate(lr_img, scale_factor=scale, mode='bicubic', align_corners=False)
        bicubic_np = lr_bicubic.squeeze().cpu().numpy()
        axes[2].imshow(bicubic_np, cmap='hot', vmin=0, vmax=1)
        axes[2].set_title(f'Standard Upscaling\n{bicubic_np.shape[0]}x{bicubic_np.shape[1]} pixels', fontsize=12, weight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()

In [8]:
def generate_client_report(total_psnr, total_ssim, num_with_gt, total_time, num_images):
    """Generate professional client report"""
    report_path = os.path.join(OUTPUT_DIR, "performance_report.txt")
    
    with open(report_path, 'w') as f:
        f.write("=" * 70 + "\n")
        f.write("THERMAL SUPER-RESOLUTION MODEL - PERFORMANCE REPORT\n")
        f.write("=" * 70 + "\n\n")
        
        f.write("📋 TEST CONFIGURATION\n")
        f.write("-" * 50 + "\n")
        f.write(f"Test Date/Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Model Checkpoint: {MODEL_DIR}\n")
        f.write(f"Test Dataset: {TEST_DIR}\n")
        f.write(f"Ground Truth: {GT_DIR if GT_DIR else 'Not provided'}\n")
        f.write(f"Output Directory: {OUTPUT_DIR}\n\n")
        
        f.write("🎯 EXECUTIVE SUMMARY\n")
        f.write("-" * 50 + "\n")
        f.write(f"Model Scale Factor: {SCALE}x resolution enhancement\n")
        f.write(f"Images Processed: {num_images} thermal images\n")
        f.write(f"Processing Speed: {1/(total_time/num_images):.1f} FPS (real-time capable)\n\n")
        
        if num_with_gt > 0:
            avg_psnr = total_psnr / num_with_gt
            avg_ssim = total_ssim / num_with_gt
            
            f.write("📊 QUALITY METRICS\n")
            f.write("-" * 50 + "\n")
            f.write(f"Peak Signal-to-Noise Ratio (PSNR): {avg_psnr:.1f} dB\n")
            f.write(f"Structural Similarity Index (SSIM): {avg_ssim:.3f}\n\n")
            
            f.write("✅ PERFORMANCE ASSESSMENT\n")
            f.write("-" * 50 + "\n")
            
            if avg_psnr > 28:
                f.write("• PSNR: EXCELLENT - Exceeds industry standards (>28 dB)\n")
            elif avg_psnr > 25:
                f.write("• PSNR: GOOD - Meets professional requirements\n")
            else:
                f.write("• PSNR: ACCEPTABLE - Basic enhancement achieved\n")
                
            if avg_ssim > 0.7:
                f.write("• SSIM: EXCELLENT - Superior detail preservation\n")
            elif avg_ssim > 0.6:
                f.write("• SSIM: GOOD - Adequate structural similarity\n")
            else:
                f.write("• SSIM: ACCEPTABLE - Basic structure maintained\n")
        
        f.write("\n🚀 DEPLOYMENT READINESS\n")
        f.write("-" * 50 + "\n")
        speed_fps = 1/(total_time/num_images)
        if speed_fps > 24:
            f.write("• Speed: REAL-TIME READY - Suitable for live applications\n")
        elif speed_fps > 10:
            f.write("• Speed: NEAR REAL-TIME - Good for most applications\n")
        else:
            f.write("• Speed: BATCH PROCESSING - Suitable for offline enhancement\n")
            
        f.write("• Memory: GPU-optimized for efficient processing\n")
        f.write("• Compatibility: CUDA-accelerated, CPU fallback available\n\n")
        
        f.write("📁 OUTPUT FILES\n")
        f.write("-" * 50 + "\n")
        f.write("• enhanced/ - AI-enhanced thermal images (production ready)\n")
        f.write("• comparisons/ - Before/after visual comparisons\n")
        f.write("• performance_report.txt - This technical summary\n\n")
        
        f.write("🎯 RECOMMENDATION\n")
        f.write("-" * 50 + "\n")
        if num_with_gt > 0 and (total_psnr / num_with_gt) > 28 and (total_ssim / num_with_gt) > 0.7:
            f.write("APPROVED FOR PRODUCTION - Model meets all quality benchmarks\n")
            f.write("and is ready for deployment in thermal imaging systems.\n")
        else:
            f.write("SUITABLE FOR DEPLOYMENT - Model provides significant\n")
            f.write("enhancement over standard upscaling methods.\n")
    
    return report_path

In [9]:
def load_model(model_path, scale, device):
    """Load trained thermal model"""
    print(f"🔄 Loading model from {model_path}")
    
    # Create model
    model = IMDN(upscale=scale, in_nc=1, out_nc=1)  # Thermal model (1 channel)
    
    # Load checkpoint
    if os.path.exists(model_path):
        checkpoint = torch.load(model_path, map_location='cpu')
        
        # Extract state dict
        if 'model_state_dict' in checkpoint:
            state_dict = checkpoint['model_state_dict']
        elif 'state_dict' in checkpoint:
            state_dict = checkpoint['state_dict']
        else:
            state_dict = checkpoint
        
        # Remove 'module.' prefix if present
        if any(key.startswith('module.') for key in state_dict.keys()):
            state_dict = {key.replace('module.', ''): value for key, value in state_dict.items()}
        
        model.load_state_dict(state_dict, strict=True)
        print("✅ Model loaded successfully")
    else:
        raise FileNotFoundError(f"Model checkpoint not found: {model_path}")
    
    model = model.to(device)
    model.eval()
    return model

In [10]:
def find_corresponding_gt(lr_path, gt_dir):
    """Find corresponding ground truth image"""
    if gt_dir is None:
        return None
    
    lr_name = Path(lr_path).stem
    # Try common extensions
    for ext in ['.png', '.jpg', '.jpeg', '.bmp']:
        gt_path = os.path.join(gt_dir, lr_name + ext)
        if os.path.exists(gt_path):
            return gt_path
    return None

In [11]:
# Setup device
device = torch.device(DEVICE if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Using device: {device}")

# Create timestamped output directory if not specified
if OUTPUT_DIR is None:
	timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
	model_name = Path(MODEL_DIR).stem
	OUTPUT_DIR = f"./results/{model_name}_{timestamp}"

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load model
model = load_model(MODEL_DIR, SCALE, device)

# Get test images
test_images = []
for ext in ['*.png', '*.jpg', '*.jpeg', '*.bmp']:
	test_images.extend(Path(TEST_DIR).glob(ext))

if not test_images:
	print(f"❌ No test images found in {TEST_DIR}")

# Limit number of samples if specified
if NUM_SAMPLES:
	test_images = test_images[:NUM_SAMPLES]

print(f"🔍 Found {len(test_images)} test images")
print(f"📊 Scale factor: {SCALE}x")
print(f"📁 Output directory: {OUTPUT_DIR}")

# Test metrics
total_psnr = 0
total_ssim = 0
total_time = 0
num_with_gt = 0

# Process each test image
for i, img_path in enumerate(tqdm(test_images, desc="Testing")):
	try:
		# Load LR image
		lr_img = load_thermal_image(str(img_path)).to(device)
		
		# Find corresponding GT image
		gt_path = find_corresponding_gt(str(img_path), GT_DIR)
		gt_img = None
		if gt_path:
			gt_img = load_thermal_image(gt_path).to(device)
		
		# Inference
		start_time = time.time()
		with torch.no_grad():
			sr_img = model(lr_img)
			# Clamp to valid range
			sr_img = torch.clamp(sr_img, 0, 1)
		inference_time = time.time() - start_time
		total_time += inference_time
		
		# Calculate metrics if GT available
		if gt_img is not None:
			# Resize GT if needed
			if gt_img.shape[-2:] != sr_img.shape[-2:]:
				gt_img = F.interpolate(gt_img, size=sr_img.shape[-2:], mode='bicubic', align_corners=False)
			
			psnr = calculate_psnr(sr_img, gt_img)
			ssim = calculate_ssim(sr_img, gt_img)
			total_psnr += psnr.item()
			total_ssim += ssim.item()
			num_with_gt += 1
		
		# Save results
		if SAVE_RESULTS:
			# Save SR image with cleaner naming
			sr_np = (sr_img.squeeze().cpu().numpy() * 255).astype(np.uint8)
			sr_save_path = os.path.join(OUTPUT_DIR, "enhanced", f"thermal_enhanced_{i+1:03d}.png")
			os.makedirs(os.path.dirname(sr_save_path), exist_ok=True)
			cv2.imwrite(sr_save_path, sr_np)
			
			# Save comparison for first 5 samples
			if i < (NUM_SAMPLES)/20:
				comp_save_path = os.path.join(OUTPUT_DIR, "comparisons", f"before_after_{i+1:03d}.png")
				os.makedirs(os.path.dirname(comp_save_path), exist_ok=True)
				save_comparison_image(lr_img, sr_img, gt_img, comp_save_path, SCALE)
	
	except Exception as e:
		print(f"❌ Error processing {img_path}: {e}")
		continue

# Generate client report
report_path = generate_client_report(total_psnr, total_ssim, num_with_gt, total_time, len(test_images))

# Print concise results
print("\n" + "="*60)
print("🎯 THERMAL ENHANCEMENT COMPLETE")
print("="*60)
print(f"📊 Processed: {len(test_images)} images")
print(f"🚀 Speed: {1/(total_time/len(test_images)):.1f} FPS")

if num_with_gt > 0:
	avg_psnr = total_psnr / num_with_gt
	avg_ssim = total_ssim / num_with_gt
	print(f"📈 Quality: PSNR {avg_psnr:.1f}dB, SSIM {avg_ssim:.3f}")
	
	# Simple quality assessment
	if avg_psnr > 28 and avg_ssim > 0.7:
		print("✅ Status: PRODUCTION READY")
	else:
		print("✅ Status: DEPLOYMENT SUITABLE")

if SAVE_RESULTS:
	print(f"\n💾 Client Deliverables:")
	print(f"   📁 {OUTPUT_DIR}/enhanced/ - Enhanced thermal images")
	print(f"   📁 {OUTPUT_DIR}/comparisons/ - Before/after samples")
	print(f"   📄 {report_path} - Technical performance report")

print("="*60)

🖥️  Using device: cpu
🔄 Loading model from /home/kronbii/repos/thermal-super-resolution/checkpoints/thermal_x2/thermal_best.pth


/home/kronbii/miniconda/envs/FLIR/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647352509/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


FileNotFoundError: Model checkpoint not found: /home/kronbii/repos/thermal-super-resolution/checkpoints/thermal_x2/thermal_best.pth